# 20231219 - Hide and Seek
Som følge av et stadig økende trusselbilde, spesielt ifra sydligere strøk, har Nordpolar sikkerhetstjeneste etablert en intern enhet som skal beskytte tjenestens egne digitale systemer mot angrep.  
Enheten består av nøye selekterte tidligere alveteknologer som har god erfaring med bekjempelse av sydpolare aktører.

Grunnet tidligere prestasjoner på Nordpolen har NISSEN selv navngitt enheten til Julens Utvalgte Lærde Elektronisk databehandlende Sikkerhets og Operative Center, forkortet JULESOC.  
JULESOCen kan blant annet bidra til å finne ondsinnede fugler i datasystemene til Julenissens verksted, grave i sildcoin transaksjoner og analyse av speilglatte kopier.

JULESOC har nylig mottatt en speilkopi av en arbeidsstasjon lokalisert på Julenissens verksted.  
Det er mistanke om at noen uautoriserte har vært inne på maskinen og tukla.  
Vi trenger at du graver frem noen spor.  
Mellomleder

- [./data/20231219-image.raw.gz](./data/20231219-image.raw.gz)


## Answer
This task was a two stage approach with a false flags embedded.
1. I used autopsy to disect the image and found the following files
    - Python source code, [./20231219/nissekodegenerator_mixed_case.py](./20231219/nissekodegenerator_mixed_case.py)
    - array of numbers, [./20231219/code_1](./20231219/code_1)
    - An image of a QR code (false flag), [./20231219/qr-code.png](./20231219/qr-code.png)
        - URL: `https://www.google.com/search?channel=fs&client=ubuntu-sn&q=YmxpbmRzcG9yIGRlc3N2ZXJyZSwgbGV0IHZpZGVyZQ%3D%3D`
        - The base64 text in the **q** parameter decodes to: `blindspor dessverre, let videre`
2. I wrote a script to extract continuous strings from the image, which additionally found 
    - a second version of the source code, which was purely lower case, [./20231219/nissekodegenerator_lowercase.py](./20231219/nissekodegenerator_lowercase.py)
    - a second, shorter, array of numbers, [./20231219/code_2](./20231219/code_2)
    - Two text files which contained a random set of words.  One with mixed case and another with just lower case.
        - [./20231219/output_text_mixed_case.txt](./20231219/output_text_mixed_case.txt)
        - [./20231219/output_text_lower_case.txt](./20231219/output_text_lower_case.txt)
3. Analysed the scripts found, both did the same but one makes all the text lower case
    - Generate a map of each characters set of locations in the text
    - For each letter in the keyword, select a random location for that character in the text
    - Print the indexes of the keyword
4. I wrote a tiny script to reverse the code which returned the flag

**FLAG:** `PST{TheGrinchWouldHateThis}`


<img src="./data/20231219-autopsy.png" alt="autopsy file discovery" width=800px />

In [ ]:
# Read all consecutive non-zero bytes of lencth 30 or more into a list
bitstreams = []
current_byte = 0
with open("./20231219/image.raw", "rb") as rawimage:
    current_bitstream = []
    while bytes_read := rawimage.read(1024):
        for byte in bytes_read:
            if byte != 0:
                current_bitstream.append(byte)
            elif current_bitstream != [] and byte == 0:
                if len(current_bitstream) >= 30:
                    bitstreams.append(current_bitstream)
                    print(f"saving bitstream: {current_byte-len(current_bitstream)}:{current_byte} - {current_bitstream[:30]}")
                current_bitstream = []
            current_byte += 1

In [ ]:
# Clean up the array, removing all sublists that starts with [255,255,255]
pop_list = []
for index,bitstream in enumerate(bitstreams):
    if bitstream[0] == 255 and bitstream[1] == 255 and bitstream[3] == 255:
        pop_list.append(index)
pop_list.sort(reverse=True)
pop_list
for pop in pop_list:
    bitstreams.pop(pop)

In [ ]:
# Save the currently longest bytesequence to file
index = bitstreams.index(max(bitstreams, key=len))
print(index)
print()
with open("20231219/raw_data.txt", "wb") as output:
    print(bitstreams[index][:30])
    for byte in bitstreams[index]:
        output.write(byte.to_bytes())
temp = bitstreams.pop(index)

In [ ]:
# Recover the flag after finding the code files, text files and decoded the scripts
indexes = [6793, 539, 4387, 0, 5815, 8316, 7006, 8628, 2750, 9710, 7513, 1344, 4841, 2172, 1949, 6157, 4629, 931, 2765, 6744, 8609, 2853, 3580, 7327, 450, 4323, 9871]
with open("20231219/output_text_mixed_case.txt") as stegfile:
    raw_text = stegfile.read()
flag = [raw_text[index] for index in indexes]
print("".join(flag))

indexes = [1817, 1004, 2238, 1709, 18, 714, 2499, 3069, 2148, 854, 1480, 831, 2441, 373, 276, 374, 844, 2725, 736, 2204, 1107, 1478]
with open("20231219/output_text_lower_case.txt") as stegfile:
    raw_text = stegfile.read()
flag = [raw_text[index] for index in indexes]
print("".join(flag))